In [7]:
import pandas as pd
import numpy as np 
import random
from sklearn import metrics
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import OrderedDict
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import calibration_curve

#plotting related things 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,  classification_report,  roc_curve, roc_auc_score
plt.rc("font", size=14)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
%matplotlib inline

In [2]:
'''
Load arrays for preping DF: 

For structure of npz:
arr one is DNA seq, arr two is label encoding, arr three is one hot 

Afterwards combine both 1 hot encoding
'''
RM_alt_seq = np.load('RM_alt_seq_Complete_V3.npz')
RM_pos_guide = np.load('RM_pos_guide_Complete_V3.npz')
RM_ref_seq = np.load('RM_ref_seq_Complete_V3.npz')

RM_alt_seq_onehot =   RM_alt_seq['arr_2']
RM_pos_guide_onehot = RM_pos_guide['arr_2']
RM_ref_seq_onehot = RM_ref_seq['arr_2']

print('shapes for alt seq, post guide,  ref:',
      RM_alt_seq_onehot.shape, RM_pos_guide_onehot.shape, RM_ref_seq_onehot.shape)

#stack arrays via cols
RM_stacks = np.column_stack((RM_alt_seq_onehot, RM_pos_guide_onehot,
                             RM_ref_seq_onehot))

Y_labels= np.load('RM_var_DF_V3.npy')
print('RM stack shape', RM_stacks.shape)
print('y label shape', Y_labels.shape)

shapes for alt seq, post guide,  ref: (23397, 81) (23397, 20) (23397, 81)
RM stack shape (23397, 182)
y label shape (23397,)


In [3]:
# 3. Split the data & targets into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(RM_stacks, Y_labels,
                                                    test_size=0.2, random_state=0)

print("Shape of  X_train dataset: ", X_train.shape)
print("Shape of  y_train dataset: ", y_train.shape)
print("Shape of  X_test dataset: ", X_test.shape)
print("Shape of  y_test dataset: ", y_test.shape)

print(" train counts of label '1': {}", (sum(y_train==1)))
print("train counts of label '0': {}", (sum(y_train==0)))

print(" test counts of label '1': {}", (sum(y_test==1)))
print("test counts of label '0': {}", (sum(y_test==0)))

Shape of  X_train dataset:  (18717, 182)
Shape of  y_train dataset:  (18717,)
Shape of  X_test dataset:  (4680, 182)
Shape of  y_test dataset:  (4680,)
 train counts of label '1': {} 249
train counts of label '0': {} 18468
 test counts of label '1': {} 57
test counts of label '0': {} 4623


# Trying with Smote 

In [4]:
print("Before OverSampling, counts of label '1': {}", (sum(y_train==1)))
print("Before OverSampling, counts of label '0': {}", (sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}", (sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}",(sum(y_train_res==0)))


Before OverSampling, counts of label '1': {} 249
Before OverSampling, counts of label '0': {} 18468
After OverSampling, the shape of train_X: (36936, 182)
After OverSampling, the shape of train_y: (36936,) 

After OverSampling, counts of label '1': {} 18468
After OverSampling, counts of label '0': {} 18468


In [41]:
# Import the model we are using
rf = RandomForestClassifier()
# Train the model on training data
rf.fit(X_train_res, y_train_res)

y_pred = rf.predict(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [42]:
print ("Train Accuracy :: ", accuracy_score(y_train_res, rf.predict(X_train_res)))
print ("Test Accuracy  :: ", accuracy_score(y_test, y_pred))

Train Accuracy ::  0.7634286333116742
Test Accuracy  ::  0.8542735042735042


In [1]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred,  normalize=False))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

NameError: name 'metrics' is not defined

In [46]:
# Import the model we are using
rf_II = RandomForestClassifier( class_weight= 'balanced')
# Train the model on training data
rf_II.fit(X_train_res, y_train_res)
y_pred_II = rf_II.predict(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [47]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_II))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred_II))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred_II))
print(classification_report(y_test, y_pred_II))


Accuracy: 0.8420940170940171
Precision: 0.015625
Recall: 0.19298245614035087
              precision    recall  f1-score   support

           0       0.99      0.85      0.91      4623
           1       0.02      0.19      0.03        57

   micro avg       0.84      0.84      0.84      4680
   macro avg       0.50      0.52      0.47      4680
weighted avg       0.98      0.84      0.90      4680



In [52]:
rf_III =RandomForestClassifier(n_estimators=100, class_weight= 'balanced',
                               warm_start=True,  oob_score=True, max_features="sqrt")

# Train the model on training data
rf_III.fit(X_train_res, y_train_res)
y_pred_III = rf_III.predict(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


In [53]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_III))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred_III))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred_III))
print(classification_report(y_test, y_pred_III))


Accuracy: 0.854059829059829
Precision: 0.013975155279503106
Recall: 0.15789473684210525
              precision    recall  f1-score   support

           0       0.99      0.86      0.92      4623
           1       0.01      0.16      0.03        57

   micro avg       0.85      0.85      0.85      4680
   macro avg       0.50      0.51      0.47      4680
weighted avg       0.98      0.85      0.91      4680



In [8]:
rf_IV =RandomForestClassifier(n_estimators=10, class_weight= 'balanced',
                               warm_start=True,  oob_score=True, max_features="sqrt")

# Train the model on training data
rf_IV.fit(X_train_res, y_train_res)
y_pred_IV = rf_IV.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_IV))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred_IV))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred_IV))
print(classification_report(y_test, y_pred_IV))



/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


Accuracy: 0.8542735042735042
Precision: 0.015503875968992248
Recall: 0.17543859649122806
              precision    recall  f1-score   support

           0       0.99      0.86      0.92      4623
           1       0.02      0.18      0.03        57

   micro avg       0.85      0.85      0.85      4680
   macro avg       0.50      0.52      0.47      4680
weighted avg       0.98      0.85      0.91      4680



/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:458: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: divide by zero encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:463: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [9]:
rf_V =RandomForestClassifier(n_estimators=50, class_weight= 'balanced',
                               warm_start=True, oob_score=True, max_features="sqrt")

# Train the model on training data
rf_V.fit(X_train_res, y_train_res)
y_pred_V = rf_V.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_V))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred_V))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred_V))
print(classification_report(y_test, y_pred_V))



/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


Accuracy: 0.854059829059829
Precision: 0.013975155279503106
Recall: 0.15789473684210525
              precision    recall  f1-score   support

           0       0.99      0.86      0.92      4623
           1       0.01      0.16      0.03        57

   micro avg       0.85      0.85      0.85      4680
   macro avg       0.50      0.51      0.47      4680
weighted avg       0.98      0.85      0.91      4680

